<a href="https://colab.research.google.com/github/kchen79427/Capstone_Bruestle_Boudia_Chen/blob/dave_branch/data_preprocessing/Data_Transformation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
import re

In [ ]:
pd.set_option('display.max_columns',150)

In [ ]:
X_ds  = pd.read_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/OnTime_w_Weather_10_19_ml_cols.parquet')
X = pd.read_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/OnTime_w_Weather_10_19.parquet')

X_what = pd.read_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/OnTime_w_Weather_23.parquet')
test = pd.read_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/OnTime_w_Weather_23_ml_cols.parquet')

In [ ]:
X_what.shape, test.shape, X.shape, X_ds.shape

((293218, 119), (56924, 61), (293218, 119), (293218, 61))

In [ ]:
list(X.columns)

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'CRSDepHour',
 'Reporting_Airline',
 'OriginAirportSeqID',
 'DestAirportSeqID',
 'Dest',
 'DestStateFips',
 'CRSElapsedTime',
 'Distance',
 'DistanceGroup',
 'mfr',
 'passengers',
 'engine_type',
 'no_engines',
 'w_dir_angle',
 'w_type',
 'w_speed_rate',
 'sky_c_hgt',
 'sky_c_det',
 'vis_dist',
 'tmp_air',
 'tmp_dew',
 'sea_lvl_p',
 'liq_precip_qty',
 'liq_precip_dim',
 'liq_precip_cond',
 'sky_cov',
 'sky_cov_base_hgt',
 'sky_cov_cld',
 'sky_sum_cov',
 'sky_sum_hgt',
 'sky_obs_tot_cov',
 'sky_low_cld_base_hgt',
 'at_pres_altimeter_rate',
 'at_pres_stn_rate',
 'w_dir_angle_d',
 'w_type_d',
 'w_speed_rate_d',
 'sky_c_hgt_d',
 'sky_c_det_d',
 'vis_dist_d',
 'tmp_air_d',
 'tmp_dew_d',
 'sea_lvl_p_d',
 'liq_precip_qty_d',
 'liq_precip_dim_d',
 'liq_precip_cond_d',
 'sky_cov_d',
 'sky_cov_base_hgt_d',
 'sky_cov_cld_d',
 'sky_sum_cov_d',
 'sky_sum_hgt_d',
 'sky_obs_tot_cov_d',
 'sky_low_cld_base_hgt_d',
 'at_pres_altimeter_rate_d',

In [ ]:
X.corr()['Weather_Label'].sort_values(ascending=False)

<ipython-input-6-210ac6352859>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  X.corr()['Weather_Label'].sort_values(ascending=False)


Weather_Label               1.000000
CRSDepHour                  0.057540
sea_lvl_p                   0.057018
tmp_dew                     0.012607
tmp_air                     0.007855
DestAirportSeqID            0.003082
passengers                  0.002188
w_speed_rate                0.000288
DestStateFips              -0.000032
sea_lvl_p_d                -0.000102
Year                       -0.000309
no_engines                 -0.001481
OriginAirportSeqID         -0.002134
at_pres_stn_rate           -0.003125
liq_precip_dim_d           -0.003617
liq_precip_qty_d           -0.004781
DayofMonth                 -0.006462
liq_precip_cond_d          -0.006658
Distance                   -0.006872
DayOfWeek                  -0.007311
sky_low_cld_base_hgt_d     -0.007367
tmp_air_d                  -0.007819
vis_dist                   -0.007873
DistanceGroup              -0.008294
CRSElapsedTime             -0.008361
at_pres_stn_rate_d         -0.008381
Month                      -0.008493
t

In [ ]:
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293218 entries, 0 to 293217
Data columns (total 61 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Year                      293218 non-null  int32 
 1   Quarter                   293218 non-null  int32 
 2   Month                     293218 non-null  int32 
 3   DayofMonth                293218 non-null  int32 
 4   DayOfWeek                 293218 non-null  int32 
 5   CRSDepHour                293218 non-null  int32 
 6   Reporting_Airline         293218 non-null  object
 7   OriginAirportSeqID        293218 non-null  int32 
 8   DestAirportSeqID          293218 non-null  int32 
 9   Dest                      293218 non-null  object
 10  DestStateFips             293218 non-null  int32 
 11  CRSElapsedTime            293218 non-null  int32 
 12  Distance                  293218 non-null  int32 
 13  DistanceGroup             293218 non-null  int32 
 14  mfr 

In [ ]:
## Script to transform dataset

# List of flight-related columns.
flight_cols = ['Year', 'Quarter', 'Month',
                  'DayofMonth', 'DayOfWeek','OriginAirportSeqID',
                  'DestAirportSeqID', 'Dest','DestStateFips',
                  'CRSElapsedTime', 'Distance',
                  'DistanceGroup']


def combine_mfr(df):
  X = df.copy()

  # This function maps to a consolidated list of manufacturers return a new DF
  X['mfr'] = df['mfr'].map({'MCDONNELL DOUGLAS':'MCDONNELL',
                            'BOEING': 'BOEING',
                            'MCDONNELL DOUGLAS AIRCRAFT CO': 'MCDONNELL',
                            'AIRBUS': 'AIRBUS','AIRBUS INDUSTRIE': 'AIRBUS',
                            'MCDONNELL DOUGLAS CORPORATION': 'MCDONNELL'
                            })

  return X


# def combine_date_time(df):
#   X = df.copy()
#   X.rename(columns={'DayofMonth':'Day'},inplace=True)
#   X['Date']=pd.to_datetime(X[['Year', 'Month', 'Day']])
#   X.drop(['Month', 'Day','Quarter','DayOfWeek','Year'], axis=1,inplace=True)
#   return X





def clean_column_names(df):
  # This cleans-up and renames the feature union's prefixes for each column
  # it first drops the categorical Dest column (which was preserved from the cols_tfr)
  X = df.copy()
  X.drop('feat__Dest',axis=1,inplace=True)
  ins_inc = 0
  new_cols = []
  cols_to_remove = []
  for col in X.columns:
      if '_99' in col:
        cols_to_remove.append(col)
      else:
        new_col = re.sub('Ordinal__','',col)
        new_col = re.sub('feat__','',new_col)
        X.rename({col: new_col},inplace=True,axis=1)
  X.drop(cols_to_remove,axis=1,inplace=True)

  # this next line basically reorders the columns so that the df starts with flight cols
  order_cols = flight_cols + [col for col in X.columns if col not in flight_cols]
  return X[order_cols]


# In order to leverage this for fit/transform we need to wrap into a Function Transformer
mfr_consolidate = (FunctionTransformer(combine_mfr)
                  .set_output(transform='pandas'))

column_cleaner = (FunctionTransformer(clean_column_names)
                  .set_output(transform='pandas'))

# dt_combiner = (FunctionTransformer(combine_date_time)
#               .set_output(transform='pandas'))




# dtype_convert = (FunctionTransformer(clean_dtypes)
#                   .set_output(transform='pandas'))

# Destinations are encoded according to number

# time_cols = ['Month', 'DayofMonth','Quarter','DayOfWeek','Year']
# dt_tfr = ColumnTransformer(transformers=[('dt_convert',
#                                           dt_combiner,
#                                           time_cols)
#                                          ],
#                            remainder='drop',
#                            verbose_feature_names_out=False)



dest_tfr = ColumnTransformer(transformers=[('dests',
                                            OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=999),
                                             ['Dest']
                                            )],
                             remainder='drop',
                             verbose_feature_names_out=False)

#
mfr_trf = ColumnTransformer(transformers=[('mfr',
                                           Pipeline([('mfr_consolidate', mfr_consolidate),
                                                     ('mfr_ohe',OneHotEncoder(sparse_output=False)
                                                     )]),
                                                    ['mfr']
                                                    )],
                             remainder='drop',
                             verbose_feature_names_out=False).set_output(transform='pandas')


# each column listed is passed to OneHotEncoding to be split into several cols
cols_tfr = ColumnTransformer([('columns_encoded',
                              OneHotEncoder(sparse_output=False),
                              ['Reporting_Airline',
                                'mfr',
                              'engine_type',
                                'w_type',
                                'sky_c_det',
                                'sky_cov',
                                'sky_obs_tot_cov',
                                'w_type_d',
                                'sky_c_det_d',
                                'sky_cov_d',
                                'sky_obs_tot_cov_d']
                                )],
                remainder='passthrough',
                verbose_feature_names_out=False).set_output(transform='pandas')


# The column transformers need to be joined together to recreate the dataframe
feat_union = FeatureUnion([
                          #  ('dt_change', dt_tfr),
                           ('Ordinal',dest_tfr),
                           ('feat', cols_tfr)
                           ]).set_output(transform='pandas')


# the pipeline to complete above processing
pipe = Pipeline([
                 ('mfr_consolidate',mfr_consolidate),
                 ('feature_union', feat_union),
                 ('col_cleaner',column_cleaner)
                ])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [ ]:
new_X = pipe.fit_transform(X)

In [ ]:
new_X.shape

(293218, 116)

In [ ]:
X_test = pipe.transform(test)

In [ ]:
new_X.to_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/Full_Training_set.parquet')

In [ ]:
# confirm file can be read in
df = pd.read_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/Training_set.parquet')

In [ ]:
# Save off test set
X_test.to_parquet('/content/drive/Shareddrives/STUDENT-Capstone SS23/BTS_data/OnTime_WithWeather/Test_set.parquet')